In [11]:
import math
from collections import defaultdict

from fts import FullTextSearch, DOCUMENTS

In [ ]:
fts = FullTextSearch()

total_length = 0

for doc in DOCUMENTS:
    doc_id = doc["id"]
    fts.documents[doc_id] = doc

    # Create a single text with title weighted more heavily
    # We repeat the title three times to give it more weight (naive version)
    text = f"{doc['title']} {doc['title']} {doc['title']} {doc['content']}"

    # Tokenize the combined text
    words = fts.tokenize(text)

    # Calculate document length
    doc_length = len(words)
    fts.doc_lengths[doc_id] = doc_length
    total_length += doc_length

    # Count word frequencies
    word_freq = defaultdict(int)
    for word in words:
        word_freq[word] += 1

    # Add to inverted index
    for word, freq in word_freq.items():
        fts.inverted_index[word].append((doc_id, freq))

fts.total_doc_count = len(DOCUMENTS)
fts.avg_doc_length = total_length / fts.total_doc_count if fts.total_doc_count > 0 else 0

fts.inverted_index
# print(fts.inverted_index['analysis'])
# print(fts.inverted_index['python'])

https://en.wikipedia.org/wiki/Okapi_BM25

![](./images/bm25_formula1.png)

In [ ]:
query = "intelligence"

query_words = fts.tokenize(query)

# BM25 parameters
k1 = 1.2  # Term frequency saturation
b = 0.75  # Length normalization

# Calculate scores for each document
doc_scores = defaultdict(float)
matched_words = defaultdict(set)

for word in query_words:
    if word not in fts.inverted_index:
        continue

    # How many documents contain this word
    docs_with_word = fts.inverted_index[word]
    docs_with_word_count = len(docs_with_word)

    # Inverse Document Frequency (IDF) - rare words are more valuable
    idf = math.log((fts.total_doc_count - docs_with_word_count + 0.5) / (docs_with_word_count + 0.5) + 1)

    for doc_id, freq in docs_with_word:
        doc_len = fts.doc_lengths[doc_id]
        numerator = freq * (k1 + 1)
        denominator = freq + k1 * (1 - b + b * (doc_len / fts.avg_doc_length))
        bm25_score = idf * (numerator / denominator)

        # Add to document score
        doc_scores[doc_id] += bm25_score
        matched_words[doc_id].add(word)

doc_scores
matched_words

In [14]:
# Prepare search results
results = []
for doc_id, score in sorted(doc_scores.items(), key=lambda x: x[1], reverse=True):
    doc = fts.documents[doc_id]
    content = doc["content"]

    results.append(
        {
            "id": doc_id,
            "title": doc["title"],
            "snippet": content[:100] + "..." if len(content) > 100 else content,
            "score": round(score, 3),
        }
    )

results

[{'id': '6',
  'title': 'Artificial Intelligence Applications',
  'snippet': 'Artificial intelligence is revolutionizing multiple industries. Healthcare uses AI for diagnosis and...',
  'score': 0.092},
 {'id': '5',
  'title': 'Advanced Computing Systems',
  'snippet': 'Artificial intelligence is transforming how systems learn and make decisions. Artificial intelligenc...',
  'score': 0.088},
 {'id': '4',
  'title': 'Artificial Intelligence Revolution in Search',
  'snippet': 'Modern computing systems are becoming increasingly sophisticated. Machine learning models can recogn...',
  'score': 0.088},
 {'id': '7',
  'title': 'Using AI for Text Search',
  'snippet': 'Artificial intelligence techniques significantly improve full text search capabilities. Modern searc...',
  'score': 0.081},
 {'id': '8',
  'title': 'Intelligent Information Retrieval',
  'snippet': 'Advanced search systems now incorporate intelligence for better results. Some elements of full text ...',
  'score': 0.072},
 {